## Setup & Imports

In [5]:
# Import required libraries
import json
import numpy as np
import warnings
from pathlib import Path
from typing import Optional

import geopandas as gpd
import matplotlib.pyplot as plt
import pandas as pd
from obspy.clients.fdsn import Client
from shapely.geometry import Point, box

from pygeohydro import WBD

#warnings.filterwarnings('ignore', category=FutureWarning)
#warnings.filterwarnings('ignore', category=UserWarning)

print("All libraries imported successfully.")

All libraries imported successfully.


In [6]:
def get_huc_geometries_local(huc_level, geom_gdf):
    """
    Fetch HUC geometries locally using USGS WBD via pygeohydro.

    Parameters
    ----------
    huc_level : str or int
        HUC level ('02','04','06','08','10','12' or 2,4,6,8,10,12)
    geom_gdf : geopandas.GeoDataFrame
        GeoDataFrame containing geometry (e.g., bbox or AOI)

    Returns
    -------
    geopandas.GeoDataFrame
        HUC geometries intersecting the input geometry
    """

    # Normalize HUC level
    huc_level = f"huc{int(huc_level)}"

    # Initialize WBD
    wbd = WBD(huc_level)

    # Extract shapely geometry (first feature)
    geom = geom_gdf.geometry.iloc[0]

    # Fetch HUCs intersecting geometry
    huc_gdf = wbd.bygeom(geom)

    return huc_gdf

## User Configuration

Basins are auto-discovered from basin names by querying USGS WBD HUC geometries using `easysnowdata`.

Notes:
- This approach uses Google Earth Engine under the hood; you may need to authenticate once.
- `bbox_wa` should tightly bound your study region to keep queries fast.

In [7]:
# ========== CONFIGURATION ==========

# List of basin names to search for
basin_names = [
    "Skagit",
    "Nooksack",
    "Skykomish",
    "Snoqualmie",
    "Cedar",  # Cedar River watershed (not Cedar Creek)
    "Green",
    "Puyallup",
    "Carbon",
    "Nisqually",
    "Cowlitz",
]

# HUC level to query (8, 10, or 12)
huc_level = 10

# FDSN client for station queries
fdsn_provider = "IRIS"

# Bounding box for western Washington (minlon, minlat, maxlon, maxlat)
# This should encompass all basins of interest
bbox_wa = (-124.8, 45.5, -120.0, 49.0)

# Optional: Filter stations by elevation (meters)
# Set to None to disable filtering
elev_min = None  # Example: 0 (sea level)
elev_max = None  # Example: 3000 (3000 meters)

# Optional: Filter by specific networks
# Set to None to include all networks
network_filter = ["UW", "CC"]  # UW + CC (as requested)

# Output directory
output_dir = Path(".")
output_dir.mkdir(exist_ok=True)

print(f"Configuration loaded:")
print(f"  HUC Level: {huc_level}")
print(f"  FDSN Provider: {fdsn_provider}")
print(f"  Bounding Box: {bbox_wa}")
print(f"  Elevation Filter: {elev_min} to {elev_max}")
print(f"  Network Filter: {network_filter}")
print(f"  Output Directory: {output_dir.absolute()}")

Configuration loaded:
  HUC Level: 10
  FDSN Provider: IRIS
  Bounding Box: (-124.8, 45.5, -120.0, 49.0)
  Elevation Filter: None to None
  Network Filter: ['UW', 'CC']
  Output Directory: /Users/marinedenolle/GitHub/gaia-data-downloaders


In [8]:
# HUC level to query (8, 10, or 12)
huc_level = 10

## Step 1: Discover HUC Codes from Basin Names

Query the USGS WBD database to find HUC codes for each basin.

In [9]:
print("🔍 Discovering HUC codes for basins...")
print(f"Searching for {len(basin_names)} basins in western Washington\n")

# HUC level string expected by easysnowdata (e.g., '08', '10', '12')
huc_level_str = f"{huc_level:02d}"
code_col = f"huc{int(huc_level_str)}"

# Create geometry from bounding box (box already imported from shapely.geometry)
bbox_geom = box(*bbox_wa)

print(f"Using HUC level: {huc_level_str} (code column: {code_col})")

🔍 Discovering HUC codes for basins...
Searching for 10 basins in western Washington

Using HUC level: 10 (code column: huc10)


In [10]:
# Warning Slow (5min+) for HUC10 over large area.
hucs_all = get_huc_geometries_local(
    huc_level=huc_level_str,
    geom_gdf=gpd.GeoDataFrame(geometry=[bbox_geom], crs="EPSG:4326"),
)
hucs_all.info()

/Users/marinedenolle/GitHub/gaia-data-downloaders/.pixi/envs/default/lib/python3.13/site-packages/pygeoogc/core.py:293: UserWarning: Found 1 failed request. Retrying ...
  resp = self._cleanup_resp(resp, payloads)


<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 288 entries, 0 to 287
Data columns (total 19 columns):
 #   Column             Non-Null Count  Dtype   
---  ------             --------------  -----   
 0   geometry           288 non-null    geometry
 1   objectid           288 non-null    int64   
 2   tnmid              288 non-null    object  
 3   metasourceid       287 non-null    object  
 4   sourcedatadesc     284 non-null    object  
 5   sourceoriginator   284 non-null    object  
 6   sourcefeatureid    0 non-null      object  
 7   loaddate           288 non-null    int64   
 8   referencegnis_ids  280 non-null    object  
 9   areaacres          288 non-null    float64 
 10  areasqkm           288 non-null    float64 
 11  states             288 non-null    object  
 12  huc10              288 non-null    object  
 13  name               288 non-null    object  
 14  hutype             288 non-null    object  
 15  humod              43 non-null     object  
 16  

In [3]:
# Filter HUCs to only those matching our prescribed basin names
name_series = hucs_all['name'].astype(str)
mask = pd.Series([False] * len(hucs_all), index=hucs_all.index)

for basin_name in basin_names:
    mask |= name_series.str.contains(basin_name, case=False, na=False)

hucs_filtered = hucs_all[mask].copy()

print(f"Filtered from {len(hucs_all)} to {len(hucs_filtered)} HUC polygons matching basin names")
print(f"Basin names searched: {basin_names}")

NameError: name 'hucs_all' is not defined

In [14]:
# Look at one to understand structure
hucs_all.iloc[0]

geometry             POLYGON ((-119.89030507051542 47.7960916326412...
objectid                                                           746
tnmid                           {D1CF8F04-E197-4E10-8B0D-DDF136689619}
metasourceid                    {511D2AC8-11BA-45FC-AB98-F69D693D4C44}
sourcedatadesc                        Watershed Boundary Dataset (WBD)
sourceoriginator     Natural Resources and Conservation Service and...
sourcefeatureid                                                   None
loaddate                                                 1723726881000
referencegnis_ids                                              1518895
areaacres                                                     131137.1
areasqkm                                                        530.69
states                                                              WA
huc10                                                       1702001203
name                                               Upper Douglas Creek
hutype

In [2]:
hucs_all.explore()

NameError: name 'hucs_all' is not defined

In [1]:
print(f"Loaded {len(hucs_all)} HUC polygon(s) in bbox.")
print('Columns:', list(hucs_all.columns))

NameError: name 'hucs_all' is not defined

In [9]:
# Build sub-basin polygons from HUCs and a dissolved basin polygon per basin_name

if 'name' not in hucs_all.columns:
    raise KeyError(f"Expected a 'name' column in HUC data. Columns: {list(hucs_all.columns)}")
if code_col not in hucs_all.columns:
    raise KeyError(f"Expected code column '{code_col}' in HUC data. Columns: {list(hucs_all.columns)}")

records = []
name_series = hucs_all['name'].astype(str)

print('Matching basin names to HUC polygons...')
for basin_name in basin_names:
    basin_hucs = hucs_all[name_series.str.contains(basin_name, case=False, na=False)].copy()
    basin_hucs[code_col] = basin_hucs[code_col].astype(str)

    if len(basin_hucs) == 0:
        print(f"  {basin_name:15s}: ⚠️ 0 HUC polygons matched")
        continue

    basin_hucs['basin_name'] = basin_name
    records.append(basin_hucs[['basin_name', 'name', code_col, 'geometry']])
    print(f"  {basin_name:15s}: ✓ {len(basin_hucs)} HUC polygon(s)")

if len(records) == 0:
    raise RuntimeError('No HUC polygons matched your basin_names. Consider adjusting basin_names or bbox_wa.')

subbasins_gdf = gpd.GeoDataFrame(pd.concat(records, ignore_index=True), crs='EPSG:4326')
subbasins_gdf = subbasins_gdf.rename(columns={'name': 'huc_name', code_col: 'huc_code'})

# Dissolve to one polygon per basin_name (for basin-level station association)
basins_gdf = subbasins_gdf.dissolve(by='basin_name', as_index=False)[['basin_name', 'geometry']]

# Attach the list of HUC codes used for each basin (string for CSV friendliness)
codes_by_basin = (
    subbasins_gdf.groupby('basin_name')['huc_code']
    .apply(lambda s: ';'.join(sorted(set(s.astype(str)))))
    .reset_index()
    .rename(columns={'huc_code': 'huc_code'})
)
basins_gdf = basins_gdf.merge(codes_by_basin, on='basin_name', how='left')

total_unique_hucs = subbasins_gdf['huc_code'].nunique()
print('-' * 70)
print(f"Matched HUC polygons: {len(subbasins_gdf)} (unique HUC codes: {total_unique_hucs})")
print('  subbasins_gdf rows:', len(subbasins_gdf))
print('  basins_gdf rows:', len(basins_gdf))
display(basins_gdf[['basin_name', 'huc_code']])
print('=' * 70)

Matching basin names to HUC polygons...
  Skagit         : ⚠️ 0 HUC polygons matched
  Nooksack       : ⚠️ 0 HUC polygons matched
  Skykomish      : ⚠️ 0 HUC polygons matched
  Snoqualmie     : ⚠️ 0 HUC polygons matched
  Cedar          : ⚠️ 0 HUC polygons matched
  Green          : ⚠️ 0 HUC polygons matched
  Puyallup       : ⚠️ 0 HUC polygons matched
  Carbon         : ⚠️ 0 HUC polygons matched
  Nisqually      : ⚠️ 0 HUC polygons matched
  Cowlitz        : ⚠️ 0 HUC polygons matched


RuntimeError: No HUC polygons matched your basin_names. Consider adjusting basin_names or bbox_wa.

## Step 2: Fetch Seismic Stations (UW, CC)

Query station metadata from the FDSN provider and convert results to a GeoDataFrame.

In [ ]:
# Fetch stations for UW + CC networks within bbox_wa
client = Client(fdsn_provider)

networks = network_filter if network_filter is not None else None
network_arg = ','.join(networks) if networks else '*'

print(f"Requesting stations from {fdsn_provider} for network={network_arg}")
inv = client.get_stations(
    network=network_arg,
    level='station',
    minlongitude=bbox_wa[0],
    minlatitude=bbox_wa[1],
    maxlongitude=bbox_wa[2],
    maxlatitude=bbox_wa[3],
)

rows = []
for net in inv:
    for sta in net:
        rows.append(
            {
                'network': net.code,
                'station': sta.code,
                'latitude': float(sta.latitude),
                'longitude': float(sta.longitude),
                'elevation_m': float(sta.elevation) if sta.elevation is not None else np.nan,
                'start_date': getattr(sta, 'start_date', None),
                'end_date': getattr(sta, 'end_date', None),
            }
)

stations_df = pd.DataFrame(rows)
if len(stations_df) == 0:
    raise RuntimeError('No stations returned. Check bbox_wa, network_filter, and fdsn_provider.')

# Optional elevation filter
if elev_min is not None:
    stations_df = stations_df[stations_df['elevation_m'] >= float(elev_min)]
if elev_max is not None:
    stations_df = stations_df[stations_df['elevation_m'] <= float(elev_max)]

stations_gdf = gpd.GeoDataFrame(
    stations_df,
    geometry=gpd.points_from_xy(stations_df['longitude'], stations_df['latitude']),
    crs='EPSG:4326',
)

print(f"Stations fetched: {len(stations_gdf)}")
display(stations_gdf.head())

## Step 3: Associate Stations with Basins/Sub-basins

Spatially join station points to dissolved basins and to sub-basin HUC polygons.

In [ ]:
# Basin-level join (dissolved polygons)
stations_basin = gpd.sjoin(
    stations_gdf,
    basins_gdf[['basin_name', 'geometry']],
    how='left',
    predicate='within',
)
stations_basin = stations_basin.drop(columns=[c for c in ['index_right'] if c in stations_basin.columns])

# Sub-basin join (HUC polygons)
stations_subbasin = gpd.sjoin(
    stations_gdf,
    subbasins_gdf[['basin_name', 'huc_code', 'huc_name', 'geometry']],
    how='left',
    predicate='within',
)
stations_subbasin = stations_subbasin.drop(columns=[c for c in ['index_right'] if c in stations_subbasin.columns])

key_cols = ['network', 'station']

# If a station falls into multiple polygons, keep the first match deterministically
stations_basin_first = stations_basin.sort_values(key_cols).drop_duplicates(subset=key_cols)
stations_subbasin_first = stations_subbasin.sort_values(key_cols).drop_duplicates(subset=key_cols)

stations_with_basins = stations_gdf.merge(
    stations_basin_first[key_cols + ['basin_name']].rename(columns={'basin_name': 'basin_name_basin'}),
    on=key_cols,
    how='left',
)
stations_with_basins = stations_with_basins.merge(
    stations_subbasin_first[key_cols + ['basin_name', 'huc_code', 'huc_name']].rename(
        columns={'basin_name': 'basin_name_subbasin'}
    ),
    on=key_cols,
    how='left',
)

# Prefer subbasin-derived basin name when present
stations_with_basins['basin_name'] = stations_with_basins['basin_name_subbasin'].combine_first(
    stations_with_basins['basin_name_basin']
 )

stations_with_basins = stations_with_basins.drop(columns=['basin_name_basin', 'basin_name_subbasin'])

assigned = stations_with_basins['basin_name'].notna().sum()
print(f"Stations assigned to a basin: {assigned} / {len(stations_with_basins)}")
display(stations_with_basins[['network','station','basin_name','huc_code','huc_name']].head(10))

## Step 4: QC Map (Basins + Stations)

Color-code each basin and plot associated stations on top for visual QA/QC.

In [ ]:
import matplotlib.patches as mpatches

# Create a stable color map for basin names
basin_list = list(basins_gdf['basin_name'])
cmap = plt.get_cmap('tab20', max(len(basin_list), 1))
basin_colors = {name: cmap(i) for i, name in enumerate(basin_list)}

fig, ax = plt.subplots(figsize=(10, 10))

# Plot basins
basins_gdf.plot(
    ax=ax,
    facecolor=basins_gdf['basin_name'].map(basin_colors),
    edgecolor='black',
    linewidth=0.8,
    alpha=0.35,
)

# Plot stations
assigned_mask = stations_with_basins['basin_name'].notna()
assigned_gdf = stations_with_basins[assigned_mask].copy()
unassigned_gdf = stations_with_basins[~assigned_mask].copy()

if len(assigned_gdf) > 0:
    assigned_gdf.plot(
        ax=ax,
        markersize=25,
        color=assigned_gdf['basin_name'].map(basin_colors),
        edgecolor='white',
        linewidth=0.4,
        zorder=5,
    )

if len(unassigned_gdf) > 0:
    unassigned_gdf.plot(
        ax=ax,
        markersize=35,
        color='black',
        marker='x',
        zorder=6,
    )

# Legend
legend_handles = [mpatches.Patch(color=basin_colors[n], label=n) for n in basin_list]
if len(unassigned_gdf) > 0:
    legend_handles.append(mpatches.Patch(color='black', label='Unassigned (outside basins)'))
ax.legend(handles=legend_handles, loc='upper right', frameon=True)

ax.set_title('Basins (dissolved) + UW/CC Stations (color-coded by basin)')
ax.set_xlabel('Longitude')
ax.set_ylabel('Latitude')

# Tighten to bbox (with a small pad)
minlon, minlat, maxlon, maxlat = bbox_wa
pad_lon = (maxlon - minlon) * 0.02
pad_lat = (maxlat - minlat) * 0.02
ax.set_xlim(minlon - pad_lon, maxlon + pad_lon)
ax.set_ylim(minlat - pad_lat, maxlat + pad_lat)
ax.set_aspect('equal', adjustable='box')

qc_path = output_dir / 'qc_basins_stations.png'
fig.savefig(qc_path, dpi=200, bbox_inches='tight')
print(f"Saved QC map: {qc_path}")
plt.show()

## Step 5: Export CSV

Write station metadata + basin/HUC assignment to a CSV file.

In [ ]:
# Export station assignments to CSV
csv_path = output_dir / 'stations_by_basin.csv'

export_cols = [
    'network',
    'station',
    'latitude',
    'longitude',
    'elevation_m',
    'basin_name',
    'huc_code',
    'huc_name',
    'start_date',
    'end_date',
]

stations_out = stations_with_basins.copy()
for c in export_cols:
    if c not in stations_out.columns:
        stations_out[c] = np.nan

stations_out = pd.DataFrame(stations_out[export_cols])
stations_out.to_csv(csv_path, index=False)
print(f"Wrote CSV: {csv_path} ({len(stations_out)} rows)")

display(stations_out.head(10))